The MIT License

Copyright (c) 2020 Yeong-Dae Kwon
 
Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:
 
The above copyright notice and this permission notice shall be included in
all copies or substantial portions of the Software.
 
 
 
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
THE SOFTWARE.


In [ ]:
SAVE_FOLDER_NAME = "INFERENCE_00"
print(SAVE_FOLDER_NAME)


In [ ]:
from HYPER_PARAMS import *  # NOTE : You much edit HYPER_PARAMS to match the model you are loading
from TORCH_OBJECTS import *
import numpy as np
import time



In [ ]:
from source.cvrp import GROUP_ENVIRONMENT
import source.MODEL__Actor.grouped_actors as A_Module
from source.utilities import Average_Meter, augment_xy_data_by_8_fold, Get_Logger


In [ ]:
# Make Log File
logger, result_folder_path = Get_Logger(SAVE_FOLDER_NAME)

print(result_folder_path)


In [ ]:
# Load Model
grouped_actor = A_Module.ACTOR().to(device)

actor_model_save_path = './result/Saved_CVRP100_Model/ACTOR_state_dic.pt'
grouped_actor.load_state_dict(torch.load(actor_model_save_path, map_location="cuda:0"))
grouped_actor.eval()

logger.info('==============================================================================')
logger.info('==============================================================================')
log_str = '  <<< MODEL: {:s} >>>'.format(actor_model_save_path)
logger.info(log_str)



In [ ]:


if PROBLEM_SIZE == 20:
    demand_scaler = 30
elif PROBLEM_SIZE == 50:
    demand_scaler = 40
elif PROBLEM_SIZE == 100:
    demand_scaler = 50
else:
    raise NotImplementedError


LOG_PERIOD_SEC = 10
DATASET_SIZE = 100 *1000


In [ ]:
TEST_BATCH_SIZE = 1024
eval_dist_AM_0 = Average_Meter()


logger.info('===================================================================')
log_str = 'Single Trajectory'
logger.info(log_str)

timer_start = time.time()
logger_start = time.time()


episode = 0
while True:
    node_data = Tensor(np.random.rand(TEST_BATCH_SIZE, PROBLEM_SIZE+1, 2))
    demand_data = Tensor(np.random.randint(1, 10, TEST_BATCH_SIZE*PROBLEM_SIZE) / demand_scaler)
    
    depot_xy = node_data[:, [0], :]
    node_xy = node_data[:, 1:, :]
    node_demand = demand_data.reshape(TEST_BATCH_SIZE, PROBLEM_SIZE, 1)
    # depot_xy.shape = (batch, 1, 2)
    # node_xy.shape = (batch, problem, 2)
    # node_demand.shape = (batch, problem, 1)

    batch_s = TEST_BATCH_SIZE
    episode = episode + batch_s
    
    

    with torch.no_grad():

        env = GROUP_ENVIRONMENT(depot_xy, node_xy, node_demand)
        group_s = 1
        group_state, reward, done = env.reset(group_size=group_s)
        grouped_actor.reset(group_state)

        # First Move is given
        first_action = LongTensor(np.zeros((batch_s, group_s)))  # start from node_0-depot
        group_state, reward, done = env.step(first_action)

        # Second Move is given
        second_action = LongTensor(np.arange(group_s))[None, :].expand(batch_s, group_s)
        group_state, reward, done = env.step(second_action)

        while not done:
            action_probs = grouped_actor.get_action_probabilities(group_state)
            # shape = (batch, group, problem+1)
            action = action_probs.argmax(dim=2)
            # shape = (batch, group)
            action[group_state.finished] = 0  # stay at depot, if you are finished
            group_state, reward, done = env.step(action)

    eval_dist_AM_0.push(-reward[:, 0])  # reward was given as negative dist



    if (time.time()-logger_start > LOG_PERIOD_SEC) or (episode >= DATASET_SIZE):
        timestr = time.strftime("%H:%M:%S", time.gmtime(time.time()-timer_start))
        log_str = 'Ep:{:07d}({:5.1f}%)  T:{:s}  avg.dist:{:f}'\
            .format(episode, episode/DATASET_SIZE*100,
                    timestr, eval_dist_AM_0.peek())
        logger.info(log_str)
        logger_start = time.time()

    if episode >= DATASET_SIZE:
        break
    
    
logger.info('---------------------------------------------------')
logger.info('average = {}'.format(eval_dist_AM_0.result()))
logger.info('---------------------------------------------------')
logger.info('---------------------------------------------------')


In [ ]:
TEST_BATCH_SIZE = 1024
eval_dist_AM_0 = Average_Meter()

logger.info('===================================================================')
log_str = 'All Trajectory'
logger.info(log_str)


timer_start = time.time()
logger_start = time.time()
episode = 0
while True:
    
    node_data = Tensor(np.random.rand(TEST_BATCH_SIZE, PROBLEM_SIZE+1, 2))
    demand_data = Tensor(np.random.randint(1, 10, TEST_BATCH_SIZE*PROBLEM_SIZE) / demand_scaler)
    
    depot_xy = node_data[:, [0], :]
    node_xy = node_data[:, 1:, :]
    node_demand = demand_data.reshape(TEST_BATCH_SIZE, PROBLEM_SIZE, 1)
    # depot_xy.shape = (batch, 1, 2)
    # node_xy.shape = (batch, problem, 2)
    # node_demand.shape = (batch, problem, 1)

    batch_s = TEST_BATCH_SIZE
    episode = episode + batch_s
    
    

    with torch.no_grad():

        env = GROUP_ENVIRONMENT(depot_xy, node_xy, node_demand)
        group_s = PROBLEM_SIZE
        group_state, reward, done = env.reset(group_size=group_s)
        grouped_actor.reset(group_state)

        # First Move is given
        first_action = LongTensor(np.zeros((batch_s, group_s)))  # start from node_0-depot
        group_state, reward, done = env.step(first_action)

        # Second Move is given
        second_action = LongTensor(np.arange(group_s))[None, :].expand(batch_s, group_s)
        group_state, reward, done = env.step(second_action)

        while not done:
            action_probs = grouped_actor.get_action_probabilities(group_state)
            # shape = (batch, group, problem+1)
            action = action_probs.argmax(dim=2)
            # shape = (batch, group)
            action[group_state.finished] = 0  # stay at depot, if you are finished
            group_state, reward, done = env.step(action)

    max_reward, _ = reward.max(dim=1)
    eval_dist_AM_0.push(-max_reward)  # reward was given as negative dist


    if (time.time()-logger_start > LOG_PERIOD_SEC) or (episode >= DATASET_SIZE):
        timestr = time.strftime("%H:%M:%S", time.gmtime(time.time()-timer_start))
        log_str = 'Ep:{:07d}({:5.1f}%)  T:{:s}  avg.dist:{:f}'\
            .format(episode, episode/DATASET_SIZE*100,
                    timestr, eval_dist_AM_0.peek())
        logger.info(log_str)
        logger_start = time.time()

    
    if episode >= DATASET_SIZE:
        break
    
logger.info('---------------------------------------------------')
logger.info('average = {}'.format(eval_dist_AM_0.result()))
logger.info('---------------------------------------------------')
logger.info('---------------------------------------------------')


In [ ]:
TEST_BATCH_SIZE = 256
eval_dist_AM_0 = Average_Meter()

logger.info('===================================================================')
log_str = 'Augmented x8 Input'
logger.info(log_str)


timer_start = time.time()
logger_start = time.time()
episode = 0
while True:
    
    node_data = Tensor(np.random.rand(TEST_BATCH_SIZE, PROBLEM_SIZE+1, 2))
    demand_data = Tensor(np.random.randint(1, 10, TEST_BATCH_SIZE*PROBLEM_SIZE) / demand_scaler)
    
    depot_xy = node_data[:, [0], :]
    node_xy = node_data[:, 1:, :]
    node_demand = demand_data.reshape(TEST_BATCH_SIZE, PROBLEM_SIZE, 1)
    # depot_xy.shape = (batch, 1, 2)
    # node_xy.shape = (batch, problem, 2)
    # node_demand.shape = (batch, problem, 1)

    batch_s = TEST_BATCH_SIZE
    episode = episode + batch_s
    

    # 8 fold Augmented
    aug_depot_xy = augment_xy_data_by_8_fold(depot_xy)
    # aug_depot_xy.shape = (8*batch, 1, 2)
    aug_node_xy = augment_xy_data_by_8_fold(node_xy)
    # aug_node_xy.shape = (8*batch, problem, 2)
    aug_node_demand = node_demand.repeat(8, 1, 1)
    # aug_node_demand.shape = (8*batch, problem, 2)
    batch_s = 8*batch_s


    with torch.no_grad():

        env = GROUP_ENVIRONMENT(aug_depot_xy, aug_node_xy, aug_node_demand)
        group_s = PROBLEM_SIZE
        group_state, reward, done = env.reset(group_size=group_s)
        grouped_actor.reset(group_state)

        # First Move is given
        first_action = LongTensor(np.zeros((batch_s, group_s)))  # start from node_0-depot
        group_state, reward, done = env.step(first_action)

        # Second Move is given
        second_action = LongTensor(np.arange(group_s))[None, :].expand(batch_s, group_s)
        group_state, reward, done = env.step(second_action)

        while not done:
            action_probs = grouped_actor.get_action_probabilities(group_state)
            # shape = (batch, group, problem+1)
            action = action_probs.argmax(dim=2)
            # shape = (batch, group)
            action[group_state.finished] = 0  # stay at depot, if you are finished
            group_state, reward, done = env.step(action)


    batch_s = round(batch_s/8)
    group_reward = reward.reshape(8, batch_s, group_s)
    # shape = (8, batch, group)
    max_reward, _ = group_reward.max(dim=2)
    max_reward, _ = max_reward.max(dim=0)
    eval_dist_AM_0.push(-max_reward)  # reward was given as negative dist



    if (time.time()-logger_start > LOG_PERIOD_SEC) or (episode >= DATASET_SIZE):
        timestr = time.strftime("%H:%M:%S", time.gmtime(time.time()-timer_start))
        log_str = 'Ep:{:07d}({:5.1f}%)  T:{:s}  avg.dist:{:f}'\
            .format(episode, episode/DATASET_SIZE*100,
                    timestr, eval_dist_AM_0.peek())
        logger.info(log_str)
        logger_start = time.time()

    
    if episode >= DATASET_SIZE:
        break
    
logger.info('---------------------------------------------------')
logger.info('average = {}'.format(eval_dist_AM_0.result()))
logger.info('---------------------------------------------------')
logger.info('---------------------------------------------------')

